In [3]:
import pandas as pd
from scapy.all import *
import os

# Load CSV file
# csv_file = "csv_packet_trace/embb_2.csv"  # Replace with your actual CSV filename
# df = pd.read_csv(csv_file)

gnb_ip = "10.42.0.1"
ue_ip = "10.42.0.31"

# Define a function to convert protocol name to Scapy class
def create_packet(row):
    src_ip = row["Source"]
    dst_ip = row["Destination"]

    # Modify IP addresses if needed
    if src_ip == "172.30.1.250":
        new_src_ip = gnb_ip
        new_dst_ip = ue_ip
    elif src_ip == "172.30.1.1":
        new_src_ip = ue_ip
        new_dst_ip = gnb_ip
    else:
        raise ValueError(f"Unknown IP address: {src_ip}")

    # Basic Ethernet and IP layer
    ip_layer = IP(src=new_src_ip, dst=new_dst_ip, proto=255)

    # Generate random payload with a size of 10 bytes
    payload = Raw(load=os.urandom(int(row["Length"] - 50))) # accounting for ip (20) and pcapdroid header (30) lengths
    pkt = ip_layer / payload

    return pkt


csv_folder = "csv_packet_trace"
pcap_folder = "tractor_pcaps"


# Iterate over all CSV files in the folder
for idx, filename in enumerate(os.listdir(csv_folder)):
    print(f"Processing file {idx + 1}/{len(os.listdir(csv_folder))}: {filename}")

    if filename.endswith(".csv"):
        csv_file = os.path.join(csv_folder, filename)
        df = pd.read_csv(csv_file)

        # Convert each row to a packet
        packets = df.apply(create_packet, axis=1)

        # Write to a PCAP file
        pcap_file = os.path.join(pcap_folder, filename.replace(".csv", ".pcap"))
        wrpcap(pcap_file, packets.tolist())

        print(f"PCAP file created: {pcap_file}")


Processing file 1/31: urllc_04_10.csv
PCAP file created: tractor_pcaps/urllc_04_10.pcap
Processing file 2/31: embb_05_18_a.csv
PCAP file created: tractor_pcaps/embb_05_18_a.pcap
Processing file 3/31: embb_04_10.csv
PCAP file created: tractor_pcaps/embb_04_10.pcap
Processing file 4/31: mmtc_04_10.csv
PCAP file created: tractor_pcaps/mmtc_04_10.pcap
Processing file 5/31: urll_2.csv
PCAP file created: tractor_pcaps/urll_2.pcap
Processing file 6/31: embb_03_03c.csv
PCAP file created: tractor_pcaps/embb_03_03c.pcap
Processing file 7/31: urllc_05_18.csv
PCAP file created: tractor_pcaps/urllc_05_18.pcap
Processing file 8/31: embb_05_18_b.csv
PCAP file created: tractor_pcaps/embb_05_18_b.pcap
Processing file 9/31: embb_03_03b.csv
PCAP file created: tractor_pcaps/embb_03_03b.pcap
Processing file 10/31: embb.csv
PCAP file created: tractor_pcaps/embb.pcap
Processing file 11/31: urll.csv
PCAP file created: tractor_pcaps/urll.pcap
Processing file 12/31: README.md
Processing file 13/31: mmtc_11_18.c